In [20]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [21]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [42]:
q =  "is it true ' The US Under Secretary of State testified that the United States supports Ukrainian biological research facilities which are institutions for scientific research to achieve scientific breakthroughs like the production of vaccines that combat major diseases. '"
#q ='" ”'
#q = 'is it true Albert Einstein said, "The only thing more dangerous than ignorance is arrogance."'
llm =  OpenAIChat(temperature=0.7)
template = """{question}\n\n"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)
question_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """Here is a statement:
{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
prompt_template = PromptTemplate(input_variables=["statement"], template=template)
assumptions_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """Here is a bullet point list of assertions:
{assertions}
For each assertion, determine whether it is true or false. If it is false, explain why.\n\n"""
prompt_template = PromptTemplate(input_variables=["assertions"], template=template)
fact_checker_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """In light of the above facts, how would you answer the question '{}'""".format(q)
template = """{facts}\n""" + template
prompt_template = PromptTemplate(input_variables=["facts"], template=template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[question_chain, assumptions_chain, fact_checker_chain, answer_chain], verbose=True)

NameError: name 'OpenAIChat' is not defined

In [72]:
print(q)
overall_chain.run(q)

is it true ' The US Under Secretary of State testified that the United States supports Ukrainian biological research facilities which are institutions for scientific research to achieve scientific breakthroughs like the production of vaccines that combat major diseases. '


> Entering new SimpleSequentialChain chain...
As an AI language model, I cannot confirm the current accuracy of this statement. However, according to a news article from the US Department of State on September 3, 2020, Under Secretary of State for Arms Control and International Security Affairs, Ambassador Bonnie Jenkins, expressed support for Ukraine's efforts to develop and strengthen its biological research capabilities during her visit to Ukraine. She also noted the importance of such facilities for the development of vaccines and treatments against infectious diseases.
1. The AI language model is not capable of independently verifying the accuracy of the statement.
2. The US Department of State issued a news ar

'It is unclear if the US Under Secretary of State specifically testified to this statement. However, it is true that the US government has expressed support for Ukrainian biological research facilities and the development of vaccines to combat major diseases.'

In [74]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate

In [72]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI


In [73]:
llm = ChatOpenAI(temperature=0)

In [74]:
from langchain.agents import tool

@tool
def check_fact_google(statement: str) -> str:
    """Returns If the  statement given as parameter is not empty 
    and returns true or false or partly false using  factchain. 
    If APIChain response is Empty then use other tool """
    llm = OpenAI(temperature=0)
    api_key = os.environ['API_KEY']
    factchain = APIChain.from_llm_and_api_docs(llm,factapidoc.GOOGLEFACT_DOCS, verbose=True)
    return factchain.run(f"Check if it is true for {statement} use  api_key like {api_key}")
   

In [ ]:
check_fact_google("Clinton, Bush, Obama, Biden and Pence all took classified documents after leaving White House")

In [32]:
tools = load_tools(["wikipedia"], llm=llm)

In [77]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.agents import tool

@tool
def check_claim_chain(claim) -> str:
    """  This tool be used to break claims in assumtions \
    Returns  Statement is final statement after fact check  \  
        The input should always be  claim as string, \
        and this function will always looks into the claim then  \
    Make a bullet point list of the assumptions \
        Take every assumtion and do fact check\
        fact check will be outside this function.
        Fact check will be done by other tool"""
    llm = ChatOpenAI(temperature=0)
    template = """{question}\n\n"""
    prompt_template = PromptTemplate(input_variables=["question"], template=template)
    question_chain = LLMChain(llm=llm, prompt=prompt_template)

    template = """Here is a statement:
    {statement}
    Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
    prompt_template = PromptTemplate(input_variables=["statement"], template=template)
    assumptions_chain = LLMChain(llm=llm, prompt=prompt_template)

    template = """Here is a bullet point list of assertions:
    {assertions}
    For each assertion, determine whether it is true or false. If it is false, explain why.\n\n"""
    prompt_template = PromptTemplate(input_variables=["assertions"], template=template)
    fact_checker_chain = LLMChain(llm=llm, prompt=prompt_template)

    template = f"""In light of the above facts, how would you answer the question '{claim}'"""
    template = """{facts}\n""" + template
    prompt_template = PromptTemplate(input_variables=["facts"], template=template)
    answer_chain = LLMChain(llm=llm, prompt=prompt_template)

    overall_chain = SimpleSequentialChain(chains=[question_chain, assumptions_chain, fact_checker_chain, answer_chain], verbose=True)
    
    return overall_chain.run(claim)

In [78]:
agent= initialize_agent(
    tools + [check_claim_chain , check_fact_google] , 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [67]:
agent("Franklin D. Roosevelt was president during ww1")



> Entering new AgentExecutor chain...
Thought: This is a historical fact-checking question, we can use Wikipedia to confirm the dates of Franklin D. Roosevelt's presidency and cross-check it with the timeline of World War 1.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Franklin D. Roosevelt presidency"
}
```

Observation: Page: Presidency of Franklin D. Roosevelt
Summary: For the presidency of Franklin D. Roosevelt, see:

Presidency of Franklin D. Roosevelt, first and second terms (1933–1937 and 1937–1941), as U.S. president
Presidency of Franklin D. Roosevelt, third and fourth terms (1941–1945 and January–April 1945), as U.S. president



Page: Presidency of Franklin D. Roosevelt, third and fourth terms
Summary: The third presidential term of Franklin D. Roosevelt began on January 20, 1941, when he was once again inaugurated as the 32nd president of the United States, and the fourth term of his presidency ended with his death on April 12, 1945. Roosevelt won a third ter

{'input': 'Franklin D. Roosevelt was president during ww1',
 'output': 'Franklin D. Roosevelt was NOT president during World War 1. He was president during World War 2.'}

In [ ]:
agent("Clinton, Bush, Obama, Biden and Pence all took classified documents after leaving White House")

In [26]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
def top5_results(query):
    return search.results(query, 5)

tool = Tool(
    name = "Google Search",
    description="Search Google for recent results.",
    func=top5_results
)

In [27]:
tool.run("Obama's first name?")

[{'title': 'Obama birth certificate, White House',
  'link': 'https://obamawhitehouse.archives.gov/sites/default/files/rss_viewer/birth-certificate-long-form.pdf',
  'snippet': "STATE OF HAWAII. 1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb. Middle Name. HUSSE. HUSSEIN. FILE. NUMBER 151."},
 {'title': 'Barack Obama - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Barack_Obama',
  'snippet': 'Barack Hussein Obama II is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party,\xa0...'},
 {'title': "What is Obama's first name? - Quora",
  'link': 'https://www.quora.com/What-is-Obamas-first-name',
  'snippet': 'His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name is Obama.'},
 {'title': 'Growing Up Barack: Meet Three Boys Named After President Obama',
  'link': 'https://www.nbcnews.com/storyline/president-obama-the-lega